In [1]:
import pandas as pd
import matplotlib
import numpy as np
import pprint as p
import re
import geopy


In [2]:
child = pd.read_csv('./data/childcare_centers.csv', low_memory=False)

In [3]:
child.head(10)

,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry
0,1,1013,Lakeshore Community Child Care Centre,Non Profit Agency,101 SEVENTH ST,M8V 3B5,3,(416) 394-7601,Public Elementary School,Seventh Street Public School,0,20,32,52,60,164,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.50419384..."
1,2,1014,Alternative Primary School Parent Group,Non Profit Agency,1100 SPADINA RD,M5N 2M6,8,(416) 322-5385,Public Elementary School,North Preparatory Public School,0,0,12,26,45,83,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.42037489..."
2,3,1015,Cardinal Leger Child Care Centre (Scarborough),Non Profit Agency,600 MORRISH RD,M1C 4Y1,25,(416) 287-0578,Catholic Elementary School,Cardinal Leger Catholic School,0,10,16,26,50,102,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.17546162..."
3,4,1016,George Brown - Richmond Adelaide Childcare Centre,Non Profit Agency,130 ADELAIDE ST W,M5H 3P5,10,(416) 415-2453,Other,NaN,10,15,40,0,0,65,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.38381242..."
4,5,1017,Woodland Nursery School (Warden Woods Communit...,Non Profit Agency,1 FIRVALLEY CRT,M1L 1N8,20,(416) 694-1138x163,High Rise Apartment,NaN,0,10,16,0,0,26,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.28102862..."
5,6,1021,Centennial College Progress Campus Child Care ...,Non Profit Agency,941 PROGRESS AVE,M1G 3T8,24,(416) 289-5000x2110,Community College/University,Centennial College Of Applied Arts & Technology,10,20,32,0,0,62,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.22702962..."
6,7,1029,Air-O-Down Child Care Centre,Non Profit Agency,7 HAWKSDALE RD,M3K 1W3,6,(416) 633-2014,Public High School,Downsview Secondary School,10,15,24,0,0,49,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.47951748..."
7,8,1031,Cedarbrae Day Nurseries (20 Greencrest),Commercial Agency,20 GREENCREST CIRCT,M1G 1B8,24,(416) 431-2504,High Rise Apartment,Greencrest Property,0,0,16,0,30,46,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.22343124..."
8,9,1037,Tiago Avenue Community Child Care,Non Profit Agency,145 TIAGO AVE,M4B 2A6,19,(416) 285-5644,Public Elementary School,Victoria Park Public School,0,0,21,0,30,51,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.29815396..."
9,10,1041,Ledbury Park Child Care Program,Non Profit Agency,95 FALKIRK ST,M5M 4K1,8,(416) 782-2539,Public Elementary School,Ledbury Park Public School,0,0,24,52,105,181,Y,13JUN22,"{'type': 'Point', 'coordinates': (-79.42718230..."


In [4]:
address = pd.read_csv('./data/ontario_addresses.csv', low_memory=False)

In [61]:
pd.set_option('display.max_columns', None)

In [44]:
pd.set_option('display.max_rows', None)

In [62]:
def pull_geometry(series):
    geometry = eval(series) # Read geometry value as dictionary
    coords = geometry['coordinates'] # Obtain coordinates from geometry value
    return {'longitude': coords[0], 'latitude': coords[1]} # Return longitude and latitude
    
def add_long_lat(df):
    geometry_dicts = df.geometry.map(pull_geometry).to_list() # Pull geometry from every value and set to list
    geometry_df = pd.DataFrame(geometry_dicts, index=df.index) # Convert list to dataframe
    df = pd.concat([df, geometry_df], axis=1) # Merge both dataframes
    return df
    
child = add_long_lat(child)

In [63]:
child.head(10)

,_id,LOC_ID,LOC_NAME,AUSPICE,ADDRESS,PCODE,ward,PHONE,bldg_type,BLDGNAME,IGSPACE,TGSPACE,PGSPACE,KGSPACE,SGSPACE,TOTSPACE,subsidy,run_date,geometry,longitude,latitude
0,1,1013,Lakeshore Community Child Care Centre,Non Profit Agency,101 SEVENTH ST,M8V 3B5,3,(416) 394-7601,Public Elementary School,Seventh Street Public School,0,20,32,52,60,164,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.50419384...",-79.504194,43.599924
1,2,1014,Alternative Primary School Parent Group,Non Profit Agency,1100 SPADINA RD,M5N 2M6,8,(416) 322-5385,Public Elementary School,North Preparatory Public School,0,0,12,26,45,83,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.42037489...",-79.420375,43.705766
2,3,1015,Cardinal Leger Child Care Centre (Scarborough),Non Profit Agency,600 MORRISH RD,M1C 4Y1,25,(416) 287-0578,Catholic Elementary School,Cardinal Leger Catholic School,0,10,16,26,50,102,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.17546162...",-79.175462,43.793288
3,4,1016,George Brown - Richmond Adelaide Childcare Centre,Non Profit Agency,130 ADELAIDE ST W,M5H 3P5,10,(416) 415-2453,Other,NaN,10,15,40,0,0,65,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.38381242...",-79.383812,43.649691
4,5,1017,Woodland Nursery School (Warden Woods Communit...,Non Profit Agency,1 FIRVALLEY CRT,M1L 1N8,20,(416) 694-1138x163,High Rise Apartment,NaN,0,10,16,0,0,26,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.28102862...",-79.281029,43.702952
5,6,1021,Centennial College Progress Campus Child Care ...,Non Profit Agency,941 PROGRESS AVE,M1G 3T8,24,(416) 289-5000x2110,Community College/University,Centennial College Of Applied Arts & Technology,10,20,32,0,0,62,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.22702962...",-79.227030,43.785273
6,7,1029,Air-O-Down Child Care Centre,Non Profit Agency,7 HAWKSDALE RD,M3K 1W3,6,(416) 633-2014,Public High School,Downsview Secondary School,10,15,24,0,0,49,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.47951748...",-79.479517,43.730868
7,8,1031,Cedarbrae Day Nurseries (20 Greencrest),Commercial Agency,20 GREENCREST CIRCT,M1G 1B8,24,(416) 431-2504,High Rise Apartment,Greencrest Property,0,0,16,0,30,46,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.22343124...",-79.223431,43.758370
8,9,1037,Tiago Avenue Community Child Care,Non Profit Agency,145 TIAGO AVE,M4B 2A6,19,(416) 285-5644,Public Elementary School,Victoria Park Public School,0,0,21,0,30,51,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.29815396...",-79.298154,43.712698
9,10,1041,Ledbury Park Child Care Program,Non Profit Agency,95 FALKIRK ST,M5M 4K1,8,(416) 782-2539,Public Elementary School,Ledbury Park Public School,0,0,24,52,105,181,Y,06JUN22,"{'type': 'Point', 'coordinates': (-79.42718230...",-79.427182,43.727425


In [49]:
def get_coord(df, index, return_dict=False):
    if return_dict:
        return {'longitude': df['longitude'][index], 'latitude': df['latitude'][index]}
    else:
        return (df['longitude'][index], df['latitude'][index])

In [25]:
type(address.loc[address.city.str.lower().str.contains('toronto') == True])

pandas.core.frame.DataFrame

In [72]:
toronto_address = address.loc[address.city.str.lower().str.contains('toronto') == True]

In [73]:
toronto_address = toronto_address.reset_index(drop=True)

In [74]:
toronto_address.to_csv('./data/toronto_addresses.csv')

In [75]:
get_coord(toronto_address, 40)

(-79.30547, 43.68524)

In [42]:
toronto_address.head()

,latitude,longitude,source_id,id,group_id,street_no,street,str_name,str_type,str_dir,unit,city,postal_code,full_addr,city_pcs,str_name_pcs,str_type_pcs,str_dir_pcs,csduid,csdname,pruid,provider
0,43.67674,-79.44749,NaN,55426498d706641c18db,3235165,1281,St Clair Ave W,NaN,NaN,NaN,NaN,former Toronto,NaN,1281 St Clair Ave W,FORMER TORONTO,ST CLAIR,AVE,W,3520005,Toronto,35,Toronto
1,43.66449,-79.32191,NaN,14fbb82f0a40ed61c5b7,3479901,39,Connaught Ave,NaN,NaN,NaN,NaN,former Toronto,NaN,39 Connaught Ave,FORMER TORONTO,CONNAUGHT,AVE,NaN,3520005,Toronto,35,Toronto
2,43.66459,-79.32195,NaN,f35ea1c4fcbeddf66025,3514798,45,Connaught Ave,NaN,NaN,NaN,NaN,former Toronto,NaN,45 Connaught Ave,FORMER TORONTO,CONNAUGHT,AVE,NaN,3520005,Toronto,35,Toronto
3,43.65130,-79.47616,NaN,4aab25f10bd7cfedda24,3384047,255,Runnymede Rd,NaN,NaN,NaN,NaN,former Toronto,NaN,255 Runnymede Rd,FORMER TORONTO,RUNNYMEDE,RD,NaN,3520005,Toronto,35,Toronto
4,43.66466,-79.44805,NaN,8046555c8a04db827f85,3322730,1B,Antler St,NaN,NaN,NaN,NaN,former Toronto,NaN,1B Antler St,FORMER TORONTO,ANTLER,ST,NaN,3520005,Toronto,35,Toronto


In [41]:
address.shape

(4033020, 22)

NameError: name 'get_distance' is not defined